<b>Objetivos<b>

Proponer una solución para distribuir 10,000 policías a pie y 1000 patrullas

Encontrar la colonia con la mayor probabilidad de, en el año 2025, ser la más peligrosa por delitos graves

Proponer un mecanismo para evaluar la estrategia de distribución de los policías a pie y patrullas

Recomendar una colonia para instalar una sucursal de una empresa, basándose en los datos

<b>Planteamiento de la solución<b>

Preparación de datos
    #Limpieza
        #Información extraviada
        #Homologar formato
        #Eliminar ruido -> valores alejados
    #Separación
        #Conjuntos de entrenamiento, evaluación, prueba (si acaso aplica)
    #Extracción de características
        #Seleccionar características que influyen en el resultado

Elección del modelo
    #Serie de tiempo ¿AR, ARMA, ARIMA, SARIMA? Elegir de acuerdo con la forma de los datos
    #Agrupamiento por k-means. A partir de qué característica se agruparán.

Entrenamiento y evaluación del modelo
    #Seleccionar parámetros iniciales (p, q, d, etc). Justificar esta selección.
    #Seleccionar número de clusters. Justificar esta selección.
    #Evaluar el modelo (score, accuracy, etc).

<b>Resultados<b>

Observaciones
    #Tendencia secular
    #Variación estacional
    #Variación cíclica
    #Variación irregular
    #¿Se detectó algún patrón?

Inferencia
    #predecir para responder a los objetivos